In [2]:
import os
import sklearn
import numpy as np
from numpy import unwrap, diff, abs, angle
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from sklearn.utils import shuffle
import scipy
from scipy.signal import butter, filtfilt, hilbert
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.layers import Dense,  BatchNormalization, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import mne
from mne.preprocessing import ICA
import pywt
from scipy.stats import skew, kurtosis
from scipy.signal import spectrogram

Preproccesing Functions

In [3]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def denoise_data(df, col_names, n_clusters):
    df_denoised = df.copy()
    for col_name, k in zip(col_names, n_clusters):
        df_denoised[col_name] = pd.to_numeric(df_denoised[col_name], errors='coerce') # Convert column to numeric format
        X = df_denoised.select_dtypes(include=['float64', 'int64']) # Select only numeric columns
        clf = KNeighborsRegressor(n_neighbors=k, weights='uniform') # Fit KNeighborsRegressor
        clf.fit(X.index.values[:, np.newaxis], X[col_name])
        y_pred = clf.predict(X.index.values[:, np.newaxis]) # Predict values 
        df_denoised[col_name] = y_pred
    return df_denoised

def z_score(df, col_names):
    df_standard = df.copy()
    for col in col_names:
        df_standard[col] = (df[col] - df[col].mean()) / df[col].std()
    return df_standard

def custom_detrend(df, col_names):
    df_detrended = df.copy()
    for col in col_names:
        y = df_detrended[col]
        x = np.arange(len(y))
        p = np.polyfit(x, y, 1)
        trend = np.polyval(p, x)
        detrended = y - trend
        df_detrended[col] = detrended
    return df_detrended

def preprocess(df, col_names, n_clusters):
    df_new = df.copy()
    df_new = denoise_data(df, col_names, n_clusters)
    # df_new = z_score(df_new, col_names)
    # df_new = custom_detrend(df_new, col_names)
    return df_new

def df_to_raw(df, sfreq=250):
    info = mne.create_info(ch_names=list(df.columns), sfreq=sfreq, ch_types=['eeg'] * df.shape[1])
    raw = mne.io.RawArray(df.T.values * 1e-6, info)  # Converting values to Volts from microvolts for MNE
    return raw

def reject_artifacts(df, channel):
    threshold_factor = 3
    median = df[channel].median()
    mad = np.median(np.abs(df[channel] - median))
    spikes = np.abs(df[channel] - median) > threshold_factor * mad
    x = np.arange(len(df[channel]))
    cs = CubicSpline(x[~spikes], df[channel][~spikes]) # Interpolate using Cubic Spline
    interpolated_values = cs(x)
    interpolated_values[spikes] *= 0.1  # Make interpolated values 0.1 times smaller
    # Check each interpolated value's difference from median and compare to the threshold
    spike_values = np.abs(interpolated_values - median) > threshold_factor * mad
    interpolated_values[spike_values] *= 0.01 
    df[channel] = interpolated_values
    return df


Define the dataset

In [4]:
folder_name = 'i'
selected_columns = ['Fz', 'FC1', 'FC2', 'C3', 'Cz', 'C4', 'CPz', 'Pz']
duration = 40 
raw=[]
event=[]
BP=[]
PP=[]
if os.path.exists(folder_name) and os.path.isdir(folder_name):
    for file_name in os.listdir(folder_name):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_name, file_name)
            s_temp = pd.read_csv(file_path, header=None)
            inst = s_temp.iloc[:, 17]
            df_temp = s_temp.iloc[:, :8]
            # print(df_temp.shape)
            # df_temp.plot(figsize=(10, 8))
            # plt.show()
            raw.append(df_temp)
            event.append(inst)
            
            # 1. Band Pass
            raw_bp = np.copy(df_temp)
            for column in range(8):
                raw_bp[:, column] = butter_bandpass_filter(raw_bp[:, column], lowcut=.4, highcut=40, fs=250) 
            print(type(raw_bp), raw_bp.shape)
            # plt.plot(raw_bp)
            # plt.show()
            
            # 2. Artifact rejection
            BP_artifact_RJ = np.copy(raw_bp)
            for channel in range (8):
                BP_artifact_RJ= reject_artifacts(pd.DataFrame(BP_artifact_RJ), channel)
            print(type(BP_artifact_RJ), BP_artifact_RJ.shape)
            # plt.plot(BP_artifact_RJ)
            # plt.show()
            
            # 3. Smoothing
            BP_artifact_RJ_SM=BP_artifact_RJ.copy()
            window_size = 10 
            for channel in range (8):
                # channel_data = BP_artifact_RJ_SM[channel, :]
                BP_artifact_RJ_SM= BP_artifact_RJ_SM.rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(method='ffill')
            # plt.plot(BP_artifact_RJ_SM)
            # plt.show()
            BP.append(BP_artifact_RJ_SM)
            
            # 4. Denoising and other preprocessing
            BP_artifact_RJ_SM.columns = selected_columns
            eeg_df_denoised = preprocess(pd.DataFrame(BP_artifact_RJ_SM), col_names=selected_columns, n_clusters=[50]*len(selected_columns))
            # eeg_df_denoised.plot(subplots=True, figsize=(15, 10), title='Denoised EEG Data')
            # plt.show()
            PP.append(eeg_df_denoised)

<class 'numpy.ndarray'> (10000, 8)
<class 'pandas.core.frame.DataFrame'> (10000, 8)
<class 'numpy.ndarray'> (10000, 8)
<class 'pandas.core.frame.DataFrame'> (10000, 8)
<class 'numpy.ndarray'> (10000, 8)
<class 'pandas.core.frame.DataFrame'> (10000, 8)
<class 'numpy.ndarray'> (10000, 8)
<class 'pandas.core.frame.DataFrame'> (10000, 8)
<class 'numpy.ndarray'> (10000, 8)
<class 'pandas.core.frame.DataFrame'> (10000, 8)
<class 'numpy.ndarray'> (10000, 8)
<class 'pandas.core.frame.DataFrame'> (10000, 8)
<class 'numpy.ndarray'> (10000, 8)
<class 'pandas.core.frame.DataFrame'> (10000, 8)
<class 'numpy.ndarray'> (10000, 8)
<class 'pandas.core.frame.DataFrame'> (10000, 8)


In [8]:
fs=250
B_N=int(len(PP)) #Number of blocks
PP_NP=np.array(PP) #shape: (B_N, 10000, 8=Channel Numbers)
event=np.array(event).reshape(B_N*(df_temp.shape[0]), 1) # df_temp.shape[0]=10000
denoised=PP_NP.reshape(B_N*(df_temp.shape[0]), 8) # seprate each blocks' signal 
pp_sig_event=np.concatenate((denoised, event), axis=1) 
labels=[] 
face = [] #lable=0
scene=[]#lable=1
# Aassuming correctness for the human behavior
for i in range(len(pp_sig_event)): #len(pp_sig_event) = the whole sample points, (df_temp.shape[0]*B_N)
    if 'M' in pp_sig_event[i, 8] or 'F' in pp_sig_event[i, 8]:
        face.append(pp_sig_event[i])
        labels.append(0)
    else:
        scene.append(pp_sig_event[i]) 
        labels.append(1)        
face = np.array(face)
scene = np.array(scene)
labels=np.array(labels) 
                 
print('event', event.shape,  'denoised',  denoised.shape, 'pp_sig_event', pp_sig_event.shape, 'face', face.shape, 'scene', scene.shape, 'labels', labels.shape)  
#denoised is all the denoised data with shape: (df_temp.shape[0]*B_N, 8)     
# event is all the events with shape: (df_temp.shape[0]*B_N, 1)                                                                                                            

event (120000, 1) denoised (120000, 8) pp_sig_event (120000, 9) face (50000, 9) scene (70000, 9) labels (120000,)


In [9]:
label=labels.reshape(int(labels.shape[0]/fs), fs)
Y=np.squeeze(label[:,0])

frequency_bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 14),
    'beta': (14, 30),
    'gamma': (30, 40),
     }

def apply_bandpass_filter(signal, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal
denoised_reshaped = denoised.reshape(int(denoised.shape[0]/250), 250, 8)

# def extract_statistical_features_from_amplitude(coefficients):
#     """Extract statistical features from the amplitude of wavelet coefficients."""
#     amplitude = np.abs(coefficients.flatten())  # Get the amplitude and flatten
#     # Extract features from amplitude
#     mean_amp = np.mean(amplitude)
#     variance_amp = np.var(amplitude)
#     skewness_amp = skew(amplitude)
#     kurtosis_amp = kurtosis(amplitude)
#     return [mean_amp, variance_amp, skewness_amp, kurtosis_amp]

# features_combined = []
# for segment in denoised_reshaped:
#     segment_features = []
#     for channel_data in segment.T:  # Going through each channel
#         # For each frequency band, apply the bandpass filter and then extract wavelet features
#         for band, (low, high) in frequency_bands.items():
#             filtered_signal = apply_bandpass_filter(channel_data, low, high, fs)
#             coefficients, frequencies = pywt.cwt(filtered_signal, scales=np.arange(1, 50), wavelet='cmor')
#             # Extract statistical features from the amplitude
#             amp_features = extract_statistical_features_from_amplitude(coefficients)
#             segment_features.extend(amp_features)            
#     features_combined.append(segment_features)
# features_combined_np = np.array(features_combined)
# print(features_combined_np.shape)
# wavelet_np=features_combined_np.reshape(features_combined_np.shape[0], 8, int(features_combined_np.shape[1]/8))

In [6]:
# Wavelet combined with frequency bands
features_combined = []

for segment in denoised_reshaped:
    segment_features = []
    for channel_data in segment.T:  # Going through each channel
        
        # For each frequency band, apply the bandpass filter and then extract wavelet features
        for band, (low, high) in frequency_bands.items():
            filtered_signal = apply_bandpass_filter(channel_data, low, high, fs)
            coeffs = pywt.wavedec(filtered_signal, 'db4', level=4)
            coeff_array = np.concatenate(coeffs)
            amplitude = np.abs(coeff_array)
            for coeff in coeffs:
                segment_features.extend([np.mean(amplitude), np.var(amplitude), skew(amplitude), kurtosis(amplitude)])
                
    features_combined.append(segment_features)    
features_combined_np = np.array(features_combined)



In [7]:
features_combined_np.shape

(480, 800)

In [8]:
# wavelet_np=features_combined_np.reshape(features_combined_np.shape[0], 8, int(features_combined_np.shape[1]/8))

# combined_features = np.concatenate([wavelet_np], axis=2)
print(features_combined_np.shape)  # Should print (1600, 8, 11)
# af=features_combined_np.reshape(int(features_combined_np.shape[0]), int(8*features_combined_np.shape[2]))
af=features_combined_np
af, Y = shuffle(af, Y)
print(af.shape, Y.shape)
# Balance the dataset
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(af, Y)
X_resampled= X_resampled.astype(np.float32)
y_resampled = y_resampled.astype(np.int32)

# Split X and y into training and testing sets
# X_touched, X_untouch, y_touch, y_untouch = train_test_split(X_resampled, y_resampled, test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled, test_size=0.1, random_state=42)

# Convert y_train and y_test to categorical format for Keras
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)
# y_untouch=tf.keras.utils.to_categorical(y_untouch, num_classes=2)


# Convert the data to a numerical type (float)
X_train = X_train.astype(np.float64)
print(X_train.shape)

# Convert one-hot-encoded labels to integer-encoded labels
y_train = np.argmax(y_train, axis=-1)
y_test = np.argmax(y_test, axis=-1)
# y_untouch = np.argmax(y_untouch, axis=-1)
print(y_train.shape, y_test.shape)


print('X_train:', X_train.shape, 'y_train:', y_train.shape, 'X_test:', X_test.shape, 'y_test:',
      y_test.shape, 'X_untouch:', 'y_untouch:' )

(480, 800)
(480, 800) (480,)
(504, 800)
(504,) (56,)
X_train: (504, 800) y_train: (504,) X_test: (56, 800) y_test: (56,) X_untouch: y_untouch:


In [14]:
mlp_data=denoised_reshaped.reshape(denoised_reshaped.shape[0], denoised_reshaped.shape[1]*denoised_reshaped.shape[2])
print(mlp_data.shape)

af_mlp=mlp_data
Y_mlp=np.squeeze(label[:,0])
af_mlp, Y_mlp= shuffle(af_mlp, Y_mlp)
print(af_mlp.shape, Y_mlp.shape)
# Balance the dataset
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled_mlp, y_resampled_mlp = oversampler.fit_resample(af_mlp, Y_mlp)
X_resampled_mlp= X_resampled_mlp.astype(np.float32)
y_resampled_mlp = y_resampled_mlp.astype(np.int32)


X_train_mlp, X_test_mlp, y_train_mlp, y_test_mlp = train_test_split(X_resampled_mlp,y_resampled_mlp, test_size=0.1, random_state=42)


(480, 2000)
(480, 2000) (480,)


In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Given data (You'd already have this loaded)
# X_train, y_train, X_test, y_test

# Ensure your target (y_train and y_test) is properly shaped.
# For binary classification, it should be of shape (n_samples, 1)
# For multi-class single-label classification, it should be one-hot encoded.

# Define the MLP model
model = Sequential()

# Input layer (with relu activation and input shape matching your feature count)
model.add(Dense(256, activation='relu', input_shape=(2000,)))

# Hidden layers
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))

# Output layer
# For binary classification, use 1 neuron with sigmoid activation
# For multi-class classification, use softmax activation and change units to number of classes
model.add(Dense(1, activation='sigmoid'))

# Compile the model
# For binary classification, use binary_crossentropy
# For multi-class classification, use categorical_crossentropy
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_mlp, y_train_mlp, epochs=100, batch_size=32, validation_data=(X_test_mlp, y_test_mlp))


Epoch 1/100
16/16 [==============================] - 1s 21ms/step - loss: 1.0425 - accuracy: 0.5099 - val_loss: 0.7461 - val_accuracy: 0.5536
Epoch 2/100
16/16 [==============================] - 0s 9ms/step - loss: 1.0569 - accuracy: 0.5536 - val_loss: 0.7303 - val_accuracy: 0.5714
Epoch 3/100
16/16 [==============================] - 0s 8ms/step - loss: 0.9711 - accuracy: 0.5615 - val_loss: 0.7363 - val_accuracy: 0.6071
Epoch 4/100
16/16 [==============================] - 0s 8ms/step - loss: 0.9201 - accuracy: 0.5417 - val_loss: 0.6414 - val_accuracy: 0.7143
Epoch 5/100
16/16 [==============================] - 0s 8ms/step - loss: 1.0369 - accuracy: 0.5198 - val_loss: 0.6935 - val_accuracy: 0.5536
Epoch 6/100
16/16 [==============================] - 0s 8ms/step - loss: 0.8816 - accuracy: 0.6131 - val_loss: 0.6427 - val_accuracy: 0.5893
Epoch 7/100
16/16 [==============================] - 0s 8ms/step - loss: 0.8755 - accuracy: 0.5437 - val_loss: 0.7035 - val_accuracy: 0.5000
Epoch 8/100


In [16]:
# Continue from the previously mentioned code
# Evaluate the model on the training set
train_loss, train_accuracy = model.evaluate(X_train_mlp, y_train_mlp, verbose=0)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_mlp, y_test_mlp, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Training Accuracy: 88.29%
Test Accuracy: 73.21%


In [21]:
import optuna
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

def objective(trial):
    model = Sequential()
    # Input layer
    n_units_i = trial.suggest_int('n_units_i', 128, 512)
    model.add(Dense(n_units_i, activation='relu', input_shape=(2000,)))
    # Hidden layers
    dropout_rate_1 = trial.suggest_float('dropout_rate_1', 0.1, 0.6)
    model.add(Dropout(dropout_rate_1))
    n_units_h1 = trial.suggest_int('n_units_h1', 32, 256)
    model.add(Dense(n_units_h1, activation='relu'))
    dropout_rate_2 = trial.suggest_float('dropout_rate_2', 0.1, 0.6)
    model.add(Dropout(dropout_rate_2))
    n_units_h2 = trial.suggest_int('n_units_h2', 16, 128)
    model.add(Dense(n_units_h2, activation='relu'))
    # Output layer
    model.add(Dense(1, activation='sigmoid'))
    # Compile the model
    lr = trial.suggest_float('lr', 1e-3, 1e-1, log=True)
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    # Train the model
    model.fit(X_train_mlp, y_train_mlp, epochs=100, batch_size=64, validation_data=(X_test_mlp, y_test_mlp), verbose=0)
    # Evaluate the model
    score = model.evaluate(X_test_mlp, y_test_mlp, verbose=0)
    return score[1]  # Return accuracy
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)
print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial
print('  Value: ', trial.value)
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')


[I 2023-10-12 21:06:19,268] A new study created in memory with name: no-name-219d32ba-677b-481a-bd7c-81d550756448


[I 2023-10-12 21:06:30,077] Trial 0 finished with value: 0.5714285969734192 and parameters: {'n_units_i': 397, 'dropout_rate_1': 0.39034802208860664, 'n_units_h1': 68, 'dropout_rate_2': 0.44598741631781813, 'n_units_h2': 67, 'lr': 0.015887083182572738}. Best is trial 0 with value: 0.5714285969734192.
[I 2023-10-12 21:06:40,231] Trial 1 finished with value: 0.6428571343421936 and parameters: {'n_units_i': 430, 'dropout_rate_1': 0.49825724466979926, 'n_units_h1': 36, 'dropout_rate_2': 0.1352095645514846, 'n_units_h2': 60, 'lr': 0.00490564738027178}. Best is trial 1 with value: 0.6428571343421936.
[I 2023-10-12 21:06:50,153] Trial 2 finished with value: 0.5892857313156128 and parameters: {'n_units_i': 386, 'dropout_rate_1': 0.22732240623546587, 'n_units_h1': 154, 'dropout_rate_2': 0.26861922306687447, 'n_units_h2': 124, 'lr': 0.025318198393738924}. Best is trial 1 with value: 0.6428571343421936.
[I 2023-10-12 21:06:58,054] Trial 3 finished with value: 0.8035714030265808 and parameters: {'

Number of finished trials:  20
Best trial:
  Value:  0.8392857313156128
  Params: 
    n_units_i: 169
    dropout_rate_1: 0.31273258780130064
    n_units_h1: 256
    dropout_rate_2: 0.4196179185478752
    n_units_h2: 97
    lr: 0.0032147738984559925


In [1]:
import optuna 
# sklearn
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU
from keras.optimizers import Adam

def objective(trial):
    model = Sequential()

    # Input layer
    n_units_i = trial.suggest_int('n_units_i', 128, 512)

    activation_choice_i = trial.suggest_categorical('activation_i', ['relu', 'leaky_relu', 'elu', 'swish'])

    if activation_choice_i == 'leaky_relu':
        model.add(Dense(n_units_i, input_shape=(2000,)))
        model.add(LeakyReLU())
    else:
        model.add(Dense(n_units_i, activation=activation_choice_i, input_shape=(2000,)))

    # Hidden layer 1
    dropout_rate_1 = trial.suggest_float('dropout_rate_1', 0.1, 0.6)
    model.add(Dropout(dropout_rate_1))

    n_units_h1 = trial.suggest_int('n_units_h1', 32, 256)
    activation_choice_h1 = trial.suggest_categorical('activation_h1', ['relu', 'leaky_relu', 'elu', 'swish'])

    if activation_choice_h1 == 'leaky_relu':
        model.add(Dense(n_units_h1))
        model.add(LeakyReLU())
    else:
        model.add(Dense(n_units_h1, activation=activation_choice_h1))

    # Hidden layer 2
    dropout_rate_2 = trial.suggest_float('dropout_rate_2', 0.1, 0.6)
    model.add(Dropout(dropout_rate_2))

    n_units_h2 = trial.suggest_int('n_units_h2', 16, 128)
    activation_choice_h2 = trial.suggest_categorical('activation_h2', ['relu', 'leaky_relu', 'elu', 'swish'])

    if activation_choice_h2 == 'leaky_relu':
        model.add(Dense(n_units_h2))
        model.add(LeakyReLU())
    else:
        model.add(Dense(n_units_h2, activation=activation_choice_h2))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    lr = trial.suggest_float('lr', 1e-3, 1e-1, log=True)
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train_mlp, y_train_mlp, epochs=100, batch_size=64, validation_data=(X_test_mlp, y_test_mlp), verbose=0)

    # Evaluate the model
    score = model.evaluate(X_test_mlp, y_test_mlp, verbose=0)
    return score[1]  # Return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial
print('  Value: ', trial.value)
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')


d:\11-software\Python3.8\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-10-13 15:13:17,467] A new study created in memory with name: no-name-ae8957c7-b9b8-4375-b807-659a871f34f7
[W 2023-10-13 15:13:20,082] Trial 0 failed with parameters: {'n_units_i': 506, 'activation_i': 'leaky_relu', 'dropout_rate_1': 0.5521514650540524, 'n_units_h1': 124, 'activation_h1': 'leaky_relu', 'dropout_rate_2': 0.5014915357712346, 'n_units_h2': 127, 'activation_h2': 'elu', 'lr': 0.001053941675509392} because of the following error: NameError("name 'X_train_mlp' is not defined").
Traceback (most recent call last):
  File "d:\11-software\Python3.8\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\tnlab\AppData\Local\Temp\ipykernel_25076\3579489060.py", line 5

NameError: name 'X_train_mlp' is not defined

In [25]:
# 1. Extract the Best Parameters
best_params = study.best_params

# 2. Build the Model using the Best Parameters
model = Sequential()

# Input Layer
n_units_i = best_params['n_units_i']

if best_params['activation_i'] == 'leaky_relu':
    model.add(Dense(n_units_i, input_shape=(2000,)))
    model.add(LeakyReLU())
else:
    model.add(Dense(n_units_i, activation=best_params['activation_i'], input_shape=(2000,)))

# Hidden Layer 1
model.add(Dropout(best_params['dropout_rate_1']))
n_units_h1 = best_params['n_units_h1']

if best_params['activation_h1'] == 'leaky_relu':
    model.add(Dense(n_units_h1))
    model.add(LeakyReLU())
else:
    model.add(Dense(n_units_h1, activation=best_params['activation_h1']))

# Hidden Layer 2
model.add(Dropout(best_params['dropout_rate_2']))
n_units_h2 = best_params['n_units_h2']

if best_params['activation_h2'] == 'leaky_relu':
    model.add(Dense(n_units_h2))
    model.add(LeakyReLU())
else:
    model.add(Dense(n_units_h2, activation=best_params['activation_h2']))

# Output Layer
model.add(Dense(1, activation='sigmoid'))

# Compile the Model
optimizer = Adam(learning_rate=best_params['lr'])
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 3. Train the Model on the Training Data
model.fit(X_train_mlp, y_train_mlp, epochs=100, batch_size=64, verbose=1)

# Evaluate the model on the training data
train_loss, train_acc = model.evaluate(X_train_mlp, y_train_mlp, verbose=1)
print("\nAccuracy on Training Data: {:.4f}".format(train_acc))


# 4. Test the Model on the Test Data
loss, accuracy = model.evaluate(X_test_mlp, y_test_mlp, verbose=0)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/100
8/8 [==============================] - 1s 7ms/step - loss: 3.7920 - accuracy: 0.5198
Epoch 2/100
8/8 [==============================] - 0s 7ms/step - loss: 3.4069 - accuracy: 0.5913
Epoch 3/100
8/8 [==============================] - 0s 7ms/step - loss: 4.1256 - accuracy: 0.5556
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: 2.8554 - accuracy: 0.5675
Epoch 5/100
8/8 [==============================] - 0s 6ms/step - loss: 2.0168 - accuracy: 0.5952
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: 1.6406 - accuracy: 0.6171
Epoch 7/100
8/8 [==============================] - 0s 7ms/step - loss: 1.1707 - accuracy: 0.6131
Epoch 8/100
8/8 [==============================] - 0s 6ms/step - loss: 0.7978 - accuracy: 0.6508
Epoch 9/100
8/8 [==============================] - 0s 6ms/step - loss: 0.7571 - accuracy: 0.6667
Epoch 10/100
8/8 [==============================] - 0s 7ms/step - loss: 0.6666 - accuracy: 0.6607
Epoch 11/100
8/8 [===========

In [22]:
# Get the best hyperparameters from the Optuna study
best_params = study.best_params

# Build the model using the best hyperparameters
model = Sequential()

model.add(Dense(best_params['n_units_i'], activation='relu', input_shape=(2000,)))
model.add(Dropout(best_params['dropout_rate_1']))
model.add(Dense(best_params['n_units_h1'], activation='relu'))
model.add(Dropout(best_params['dropout_rate_2']))
model.add(Dense(best_params['n_units_h2'], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=best_params['lr'])
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using the training data
model.fit(X_train_mlp, y_train_mlp, epochs=100, batch_size=64, verbose=1)

# Evaluate the model on the training data
train_loss, train_acc = model.evaluate(X_train_mlp, y_train_mlp, verbose=1)
print("\nAccuracy on Training Data: {:.4f}".format(train_acc))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test_mlp, y_test_mlp, verbose=1)
print("\nAccuracy on Test Data: {:.4f}".format(test_acc))


Epoch 1/100
8/8 [==============================] - 1s 10ms/step - loss: 1.2282 - accuracy: 0.5317
Epoch 2/100
8/8 [==============================] - 0s 9ms/step - loss: 1.1109 - accuracy: 0.5893
Epoch 3/100
8/8 [==============================] - 0s 8ms/step - loss: 1.2039 - accuracy: 0.5575
Epoch 4/100
8/8 [==============================] - 0s 8ms/step - loss: 0.9232 - accuracy: 0.5952
Epoch 5/100
8/8 [==============================] - 0s 9ms/step - loss: 0.9003 - accuracy: 0.5933
Epoch 6/100
8/8 [==============================] - 0s 8ms/step - loss: 0.8236 - accuracy: 0.5913
Epoch 7/100
8/8 [==============================] - 0s 8ms/step - loss: 0.7835 - accuracy: 0.5754
Epoch 8/100
8/8 [==============================] - 0s 8ms/step - loss: 0.6992 - accuracy: 0.6508
Epoch 9/100
8/8 [==============================] - 0s 8ms/step - loss: 0.6407 - accuracy: 0.6706
Epoch 10/100
8/8 [==============================] - 0s 8ms/step - loss: 0.6500 - accuracy: 0.6647
Epoch 11/100
8/8 [==========

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Function to define and compile the model based on hyperparameters
def create_model(n_units_i, dropout_rate_1, n_units_h1, dropout_rate_2, n_units_h2, lr):
    model = Sequential()
    
    # Input layer
    model.add(Dense(n_units_i, activation='relu', input_shape=(2000,)))

    # Hidden layers
    model.add(Dropout(dropout_rate_1))
    model.add(Dense(n_units_h1, activation='relu'))
    model.add(Dropout(dropout_rate_2))
    model.add(Dense(n_units_h2, activation='relu'))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Using the best hyperparameters found by Optuna
best_params = study.best_params
best_model = create_model(
    best_params['n_units_i'], 
    best_params['dropout_rate_1'], 
    best_params['n_units_h1'], 
    best_params['dropout_rate_2'], 
    best_params['n_units_h2'], 
    best_params['lr']
)

# Train the model using the entire training dataset
best_model.fit(X_train_mlp, y_train_mlp, epochs=50, batch_size=64, validation_data=(X_test_mlp, y_test_mlp))

# Evaluate the model using the test dataset
loss, accuracy = best_model.evaluate(X_test_mlp, y_test_mlp)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")


Epoch 1/50
8/8 [==============================] - 1s 27ms/step - loss: 3.3977 - accuracy: 0.4841 - val_loss: 2.1189 - val_accuracy: 0.4464
Epoch 2/50
8/8 [==============================] - 0s 12ms/step - loss: 2.0230 - accuracy: 0.5099 - val_loss: 1.4850 - val_accuracy: 0.5000
Epoch 3/50
8/8 [==============================] - 0s 12ms/step - loss: 1.3757 - accuracy: 0.5417 - val_loss: 1.4836 - val_accuracy: 0.5179
Epoch 4/50
8/8 [==============================] - 0s 11ms/step - loss: 0.9627 - accuracy: 0.5317 - val_loss: 0.6867 - val_accuracy: 0.5357
Epoch 5/50
8/8 [==============================] - 0s 11ms/step - loss: 0.7598 - accuracy: 0.5357 - val_loss: 0.6881 - val_accuracy: 0.5000
Epoch 6/50
8/8 [==============================] - 0s 11ms/step - loss: 0.6915 - accuracy: 0.5754 - val_loss: 0.7572 - val_accuracy: 0.5893
Epoch 7/50
8/8 [==============================] - 0s 11ms/step - loss: 0.7104 - accuracy: 0.5853 - val_loss: 0.6468 - val_accuracy: 0.6964
Epoch 8/50
8/8 [===========

In [214]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize and train a Random Forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict class labels for the test set
y_pred = clf.predict(X_test)

# Generate and print classification report
report = classification_report(y_test, y_pred)
print(report)

# Print accuracy score
print("Accuracy:", accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.50      0.74      0.60        23
           1       0.73      0.48      0.58        33

    accuracy                           0.59        56
   macro avg       0.61      0.61      0.59        56
weighted avg       0.63      0.59      0.59        56

Accuracy: 0.5892857142857143


In [215]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Define the objective function
def objective(trial):
    # Define search space
    n_estimators = trial.suggest_int('n_estimators', 100, 150)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    class_weight = trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample'])

    clf = RandomForestClassifier(
        n_estimators=n_estimators, 
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42,
        criterion=criterion,
        class_weight= class_weight
    )

    # Use StratifiedKFold cross-validation
    stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    return cross_val_score(clf, X_train, y_train, cv=stratified_kfold, n_jobs=-1).mean()

# Create a study object and specify the direction is maximize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Best hyperparameters
print(study.best_trial.params)




[I 2023-10-09 19:19:29,750] A new study created in memory with name: no-name-3beb3da4-ae0f-4191-a27b-e0a0b293ea23
[I 2023-10-09 19:19:30,921] Trial 0 finished with value: 0.5993267326732674 and parameters: {'n_estimators': 127, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': 'auto', 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5993267326732674.
[I 2023-10-09 19:19:31,377] Trial 1 finished with value: 0.5952079207920793 and parameters: {'n_estimators': 110, 'min_samples_split': 4, 'min_samples_leaf': 7, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5993267326732674.
[I 2023-10-09 19:19:31,880] Trial 2 finished with value: 0.5952871287128714 and parameters: {'n_estimators': 142, 'min_samples_split': 9, 'min_samples_leaf': 9, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5993267326732674.
[I 2023-10-09 19:19:32,394] 

{'n_estimators': 120, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced'}


In [216]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Splitting the data
# X_train_split, X_val, y_train_split, y_val = train_test_split(
#     X_train, y_train, test_size=0.2, random_state=42)

# Use best hyperparameters from the Optuna optimization (replace these with actual best params)
best_params = study.best_trial.params

clf_best = RandomForestClassifier(**best_params, random_state=42)
clf_best.fit(X_train, y_train)

# Predict on the validation set
y_pred = clf_best.predict(X_test)

# Generate and print classification report
report = classification_report(y_test, y_pred)
print(report)

# Print accuracy score
print("Accuracy:", accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.50      0.78      0.61        23
           1       0.75      0.45      0.57        33

    accuracy                           0.59        56
   macro avg       0.62      0.62      0.59        56
weighted avg       0.65      0.59      0.58        56

Accuracy: 0.5892857142857143


In [217]:
from sklearn import svm
from joblib import dump
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
# Create a linear SVM classifier
clf = svm.SVC(kernel='linear')

# Train the classifier
clf.fit(X_train, y_train)
# Save the model to disk
filename = 'C:/Users/tnlab/OneDrive/Documents/GitHub/Neurofeedback-Based-BCI/SVM for Unicorn Data/my_svm_model.joblib'
dump(clf, filename)

# Make predictions on the test set
y_pred = clf.predict(X_test)

print('Model accuracy: ', accuracy_score(y_test, y_pred))
report_svm_matrix = classification_report(y_test, y_pred)
print("Classification Report:")
print(report_svm_matrix)
report_svm = classification_report(y_test, y_pred, output_dict=True)

report_df_svm = pd.DataFrame(report_svm).transpose()
report_df_svm.to_excel(f"svm_classification_report_{folder_name}.xlsx", index=True)

Model accuracy:  0.5357142857142857
Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.57      0.50        23
           1       0.63      0.52      0.57        33

    accuracy                           0.54        56
   macro avg       0.54      0.54      0.53        56
weighted avg       0.56      0.54      0.54        56



In [86]:
import optuna
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Define the objective function
def objective(trial):
    # Define search space
    C = trial.suggest_loguniform('C', 1e-10, 1e10)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 1, 5)
    else:
        degree = 1  # default value, won't be used
    coef0 = trial.suggest_float('coef0', -1.0, 1.0)
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

    # Create SVM classifier
    clf = SVC(C=C, kernel=kernel, degree=degree, coef0=coef0, gamma=gamma, random_state=42)

    # Use StratifiedKFold cross-validation
    stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    return cross_val_score(clf, X_train, y_train, cv=stratified_kfold, n_jobs=-1).mean()

# Create a study object and specify the direction is maximize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Best hyperparameters
print(study.best_trial.params)


[I 2023-10-09 18:02:31,953] A new study created in memory with name: no-name-883a66fd-3ec7-44b3-b337-2e21922ac9f2
C:\Users\tnlab\AppData\Local\Temp\ipykernel_16848\3781216325.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-10, 1e10)
[I 2023-10-09 18:02:32,028] Trial 0 finished with value: 0.6566336633663367 and parameters: {'C': 37736667.2223723, 'kernel': 'rbf', 'coef0': 0.8298523640508921, 'gamma': 'scale'}. Best is trial 0 with value: 0.6566336633663367.
C:\Users\tnlab\AppData\Local\Temp\ipykernel_16848\3781216325.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-10, 1e10)
